In [12]:
%run ../talktools.py

<center>
<h1>Python Application Building</h1>
<img src="http://i.imgur.com/91PUPZA.png" width=20%>
</center>

<center>
AY 250, 2013-2022 Josh Bloom
</center>

### Today 

1. Application Building
2. Testing & Debugging
3. Structuring Code for Release and Continuous Integration
4. Hosting applications in the Cloud

[PyPI](https://pypi.org/) is the main package repository for Python

In [ ]:
!pip install requests --target=/tmp/

In [ ]:
!ls /tmp/requests*

`...dist-info` directory is part of the "standard infrastructure to manage project distributions installed on a system, so all tools that are installing or removing projects are interoperable." [PEP-376](https://www.python.org/dev/peps/pep-0376/#id19)

[Note the .whl extension...see ["Wheel vs EGG"](https://packaging.python.org/en/latest/discussions/wheel-vs-egg/) and [Wheel info](https://wheel.readthedocs.io/en/latest/)]

Now, you can have Python know about your special installation directory by modifying your `PYTHONPATH` environment variable in your `.bashrc`, `.cshrc`, or `.tcshrc` file:
```bash
#BASH Style: 
export PYTHONPATH=/tmp/requests:$PYTHONPATH
#CSH Style:
setenv PYTHONPATH /path/to/my_choice:$PYTHONPATH
```

# Getting and Installing Packages with `pip`/`setup.py` #

Sometimes `conda` and `pip` cannot find a codebase you're trying to install. In this case you'll need to do it yourself using a tarball and a `setup.py` file. This is the most straightforward way to get packages: download them from the developer’s website and hope that they’ve followed the standard conventions. Eg. [requests](https://github.com/psf/requests)

There is a standard Python package distribution scheme using `setuptools` and `setup.py` files...more on that later.

Basic workflow of installing a package with `setup.py`:

```bash
$ cd [folder with package and setup.py file]
$ [sudo] pip install  .
   # [ progress report ... ]
$ Finished processing dependencies for [package]
   # [if you want more info, there are several options to modify]
$ pip install --help
```

To do a custom installation directory (if you dont have sudo, e.g.):

```bash
$ pip install  . --prefix=/tmp/mypy/
```

Useful for debugging: to have access to this package in your PYTHONPATH while still editing it:

```bash
$ pip install -e .
```


# Managing Packages - `venv`/`conda` environments #

* Open Source software is constantly changing - how do you protect working code against future updates?
* Or, what if there is a beta release of a package you want to try, but you don’t want to fully commit yet?
* `venv` and `conda -n` creates a local, self-contained, and totally separate python installation.
* Use it to create a local Python ecosystem, separate from your computer’s main system, so that you can do what you want in one without affecting the other.

# `venv` #

<quote>venv (for Python 3) allow you to manage separate package installations for different projects. They essentially allow you to create a “virtual” isolated Python installation and install packages into that virtual installation. When you switch projects, you can simply create a new virtual environment and not have to worry about breaking the packages installed in the other environments. It is always recommended to use a virtual environment while developing Python applications.</quote>

cf. https://packaging.python.org/en/latest/guides/installing-using-pip-and-virtual-environments/#creating-a-virtual-environment

Creating a new environment with venv:

[![asciicast](https://asciinema.org/a/466731.svg)](https://asciinema.org/a/466731)

(https://asciinema.org/)



During a shell session, you can source this environment so that it runs as the default:

```bash
$ source class3_env/bin/activate
(class3_env)$
#[ pip and python commands now point to new environment ]
(class3_env)$ which python
class3_env/bin/python
```

We can save all the install packages and use those for later:

```bash
$ pip freeze > requirements.txt
$ cat requirements.txt

certifi==2021.10.8
charset-normalizer==2.0.11
idna==3.3
numpy==1.22.2
pandas==1.3.5
python-dateutil==2.8.2
pytz==2021.3
requests==2.27.1
six==1.16.0
urllib3==1.26.8
```

We can get out of the environment:

```bash
 deactivate
```
Just delete to remove environment:

```bash
rm -r class3_env
```


# conda #

https://www.anaconda.com/products/individual

In [ ]:
!conda info

In [ ]:
!conda info -e

In [ ]:
!conda search requests

In [ ]:
!conda create -n requests-test requests=2.27.1 -y

In [ ]:
!conda info -e

In [ ]:
!ls $HOME/miniconda3/envs/requests-test/bin

We could make this environment the default if we want to:
```bash
export PATH=~/miniconda3/envs/requests-test/bin:$PATH
```
And if we want to remove that environment:
```bash
conda remove -n requests-test --all
```

In [ ]:
!conda remove -n requests-test --all -y

In [ ]:
!conda info -e

## Poetry

[poetry](https://python-poetry.org/) Modern package mangement, building, publishing, environment management in one.

```bash
pip install poetry
cd /path/to/your/project
pip init
```

Everything is managed with the `pyproject.toml` file.

<center><h1> Command Line Parsing</h1></center>

```bash
python myawesomeprogram.py -o option1 -p parameter2 -Q -R
```

**Goal**: build a command-line 'standalone' codebase in Python, w/ CL options & keywords
 
 **Solution**: `argparse` (https://docs.python.org/3/library/argparse.html)
 
* Allows for  user-friendly command line interfaces, and leaves it up to the code to determine what it was the user wanted.

* Also automatically generates help & usage messages and issues errors when invalid arguments are provided.

(Note on `optparse`: being replaced in favor of `argparse`)

In [ ]:
import argparse

# Setting up a parser #


* First step for `argparse`: create parser object & tell it what arguments to expect. 
* It can then be used to process the command line arguments on runtime
* Parser class: `ArgumentParser`. Takes several arguments to set up the description used in the help text for the program & other global behaviors 
   
 <p>
See  http://www.doughellmann.com/PyMOTW/argparse/
</p>

In [ ]:
%%writefile myfile.py
#!/usr/bin/env python
import argparse
parser = argparse.ArgumentParser(description='Sample Application')
print("hi")

In [ ]:
%run myfile.py

In [ ]:
!chmod 700 myfile.py

In [ ]:
!./myfile.py

# Defining Arguments & Parsing

* Arguments can trigger different actions, specified by the action argument to `add_argument()`. 
* Several supported actions.
* Once all of the arguments are defined, you can parse the command line by passing a sequence of argument strings to `parse_args()`. 
* By default, arguments are taken from `sys.argv[1:]`, but you can also pass your own list.

In [ ]:
%%file argparse_action.py
import argparse
parser = argparse.ArgumentParser(description='Sample Application')
parser.add_argument('required_arg_1', help='This positional argument is required')
parser.add_argument('required_arg_2', help='This positional argument is also required')
parser.add_argument('-s', action='store', dest='simple_value',
                    help='Store a simple value')
parser.add_argument('-c', action='store_const', dest='constant_value',
                    const='value-to-store',
                    help='Store a constant value')
parser.add_argument('-t', action='store_true', default=False,
                    dest='boolean_switch',
                    help='Set a switch to true')
parser.add_argument('-a', action='append', dest='collection',
                    default=[],
                    help='Add repeated values to a list',
                    )
parser.add_argument('-A', action='append_const', dest='const_collection',
                    const='value-1-to-append',
                    default=[],
                    help='Add different values to list')
parser.add_argument('-B', action='append_const', dest='const_collection',
                    const='value-2-to-append',
                    help='Add different values to list')
parser.add_argument('--version', action='version', version='%(prog)s 1.0')

results = parser.parse_args()
print('required_args    =', results.required_arg_1, results.required_arg_2)
print('simple_value     =', results.simple_value)
print('constant_value   =', results.constant_value)
print('boolean_switch   =', results.boolean_switch)
print('collection       =', results.collection)
print('const_collection =', results.const_collection)

In [ ]:
%run argparse_action.py --help

* `store`: Save the value, after optionally converting it to a different type (default)
* `store_const`: Save the value as defined as part of the argument specification, rather than a value that comes from the arguments being parsed
* `store_true`/`store_false`: Save the appropriate boolean value
* `append`: Save the value to a list.  Multiple values are saved if the argument is repeated
* `append_const`: Save a value defined in the argument specification to a list
* `version`: Prints version details about the program and then exits

There's also some other ways of doing argument parsing that you might want to explore:

 * **Fire**: https://github.com/google/python-fire/blob/master/docs/guide.md
 * **Click**: https://click.palletsprojects.com/en/7.x/quickstart/


In [ ]:
#!pip install click fire

In [ ]:
%%writefile hello-click.py
import click
@click.command()
@click.option('--count', default=1, help='number of greetings')
@click.argument('name')
def hello(count, name):
    for x in range(count):
        click.echo('Hello, %s!' % name)

if __name__ == '__main__':
    hello()

In [ ]:
!python hello-click.py --help

In [ ]:
!python hello-click.py --count=4 Cal

In [ ]:
%%writefile hello-fire.py
import fire

def hello(count, name):
    "number of greetings"
    for x in range(count):
        print('Hello, %s!' % name)

if __name__ == '__main__':
    fire.Fire(hello)

In [ ]:
!python hello-fire.py --help

In [ ]:
!python hello-fire.py --count=3 "AY250 Class"

# Breakout! #

* Go to the breakout folder in: `../Breakouts/02_Versioning_Application_Building/`

* Work on the file `breakout1.py`.  Do not move or modify the other files, in the other folders, but you will need to use them.  (You may add files to these directories, if necessary)

* Build up a command line parser which allows the user to specify:
 - how many datapoints to generate
 - whether to plot with a filled in histogram or an outlined one
 - the title of the plot
 - And then have the plot be generated.

* We want to be able to run a command like:

```bash
python breakout1.py -t -n 200 -T "My Awesome Title"
```

In [ ]:
cd ~/Classes/python-seminar/Breakouts/02_Versioning_Application_Building/sol

In [ ]:
!cat plotting/histOutline.py

In [ ]:
%matplotlib inline

In [ ]:
%run breakout1_solution.py -t

In [ ]:
%run breakout1_solution.py -n 200 -T "My Awesome Title"

In [ ]:
%run breakout1_solution.py --help

In [13]:
%load_ext watermark

In [14]:
%watermark --iversions

json    : 2.0.9
argparse: 1.1
autopep8: 1.6.0
numpy   : 1.20.3

